In [1]:
import numpy as np
from pynq import Overlay, Xlnk, allocate

np.set_printoptions(suppress=True)

In [2]:
x = np.loadtxt("wave.txt", dtype=np.float32)
TWO_TO_THE_POWER_OF_N_MAX = 10
NFFT = 1 << TWO_TO_THE_POWER_OF_N_MAX

In [3]:
overlay = Overlay("fft_float_complex_pynq_z1.bit")
in_buffer = allocate(shape=(NFFT,), dtype=np.csingle)
out_buffer = allocate(shape=(NFFT,), dtype=np.csingle)
data = x[:NFFT].astype(np.csingle)

#overlay = Overlay("fft_float_pynq_z1.bit")
#in_buffer = allocate(shape=(NFFT,), dtype=np.float32)
#out_buffer = allocate(shape=(NFFT*2,), dtype=np.float32)
#data = x[:NFFT].astype(np.float32)

fft_ip = overlay.hls_fft_0
dma_ip = overlay.axi_dma_0

fft_ip.write(0x10, TWO_TO_THE_POWER_OF_N_MAX)

In [4]:
def fft_on_hw(fft_ip, dma_ip, input_data):    
    Xlnk.cma_memcopy(in_buffer, input_data, in_buffer.data.nbytes)
    dma_ip.sendchannel.transfer(in_buffer)
    dma_ip.recvchannel.transfer(out_buffer)
    fft_ip.write(0x00, 0x01)
    while not ((fft_ip.read(0x00) >> 1) & 0x01):
        continue
    dma_ip.sendchannel.wait()
    dma_ip.recvchannel.wait() 
    return out_buffer.copy()

In [5]:
fft_on_hw(fft_ip, dma_ip, data)

PynqBuffer([ 11.94231319+0.j        ,  11.95206070-0.15409547j,
             11.98142624-0.30866641j, ...,  12.03069210+0.46418774j,
             11.98143768+0.30866188j,  11.95206738+0.15408981j], dtype=complex64)